## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [3]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

--2020-11-10 15:21:57--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Risoluzione di www.dropbox.com (www.dropbox.com)... 162.125.69.1, 2620:100:6025:1::a27d:4501
Connessione a www.dropbox.com (www.dropbox.com)|162.125.69.1|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 301 Moved Permanently
Posizione: /s/raw/xvjzaxzz3ysphme/data_000637.txt [segue]
--2020-11-10 15:21:57--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Riutilizzo della connessione esistente a www.dropbox.com:443.
Richiesta HTTP inviata, in attesa di risposta... 302 Found
Posizione: https://ucfdd226d66efde2398815e7430a.dl.dropboxusercontent.com/cd/0/inline/BC43DJ1kmarGJKr8G_joqhGtTl8mTv71xSkHmCy0KeHfATExezqUWvtQ9fEwUtiYYF-1AyYoTc8vH6iXinkWMK4ckO05ttcRpZUDCuppap9-ClvYGQ5vmr_aoTTVXNKvZXA/file# [segue]
--2020-11-10 15:21:58--  https://ucfdd226d66efde2398815e7430a.dl.dropboxusercontent.com/cd/0/inline/BC43DJ1kmarGJKr8G_joqhGtTl8mTv71xSkHmCy0KeHfATExezqUWvtQ9fEwUtiYYF-1AyYoTc8v

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. 
#Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

N = 50000

data = pd.read_csv('/home/sabrina/data/data_000637.txt')
df = data.loc[0:N-1]
df



,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
49995,1,1,2,3869200664,1103,4
49996,1,1,4,3869200664,1105,14
49997,1,0,59,3869200664,1191,17
49998,1,0,32,3869200664,1191,1


In [53]:
#2. Find out the number of BX in a ORBIT (the value 'x').

#print (df.iloc[:,3])

data.groupby('ORBIT_CNT')['BX_COUNTER'].sum()

ORBIT_CNT
3869200167    124133
3869200168     97201
3869200169    144343
3869200170    217462
3869200171    228822
               ...  
3869211167    361697
3869211168    205302
3869211169    311322
3869211170    277743
3869211171      7573
Name: BX_COUNTER, Length: 11001, dtype: int64

In [ ]:
#3. Find out how much the data taking lasted. You can either make an estimate based on the fraction 
#of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.




In [25]:
#4. Create a new column with the absolute time in ns
#(as a combination of the other three columns with timing information).


#abs_time = 

#abs_time = ['1','2']

#df['ABSOLUTE_TIME (ns)'] = abs_time

In [42]:
#5. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

#df_random = df.loc[df['HEAD'] = random.randrange(0,1)]
#df_random

import random
#df.loc[df['HEAD'] == 1, 'HEAD'] =  2
#df

In [46]:
#6. Create a new DataFrame that contains only the rows with HEAD=1.


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS


In [ ]:
#7. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel


In [22]:
#8. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)


In [ ]:
#9. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139